In [15]:
from load_data import  sim_arr

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split


# Split the data into train, test and validation sets
train_set, test_set = train_test_split(sim_arr, test_size=0.2, random_state=42)
train_set, valid_set = train_test_split(
    train_set, test_size=0.2, random_state=42)

print("Train set size: ", len(train_set),
      " Test set size: ", len(test_set),
      " Validation set size: ", len(valid_set))

Train set size:  1926  Test set size:  602  Validation set size:  482


In [17]:
# check gpu
import tensorflow as tf
import os


print("Num GPUs Available: ", len(
    tf.config.experimental.list_physical_devices('GPU')))

os.environ['XLA_FLAGS'] = './cuda_sdk_lib/'

Num GPUs Available:  1


In [18]:
# check gpu
import tensorflow as tf
import os


print("Num GPUs Available: ", len(
    tf.config.experimental.list_physical_devices('GPU')))

os.environ['XLA_FLAGS'] = './cuda_sdk_lib/'

Num GPUs Available:  1


In [25]:
import optuna

from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Conv2DTranspose, Flatten
from tensorflow.keras.models import Sequential
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# get input shape from the data
input_shape = train_set.shape[1:]
output_shape = (1, 1, 2)


def create_model(trial):
    # Hyperparameters to optimize
    num_layers = trial.suggest_int('num_layers', 1, 4)

    filters = [trial.suggest_int(f'filters_{i}', 1, 12)
               for i in range(num_layers -1)] + [output_shape[-1]]

    activation = [trial.suggest_categorical(
        f'activation_{i}', ['relu', 'selu', 'tanh', 'sigmoid']) for i in range(num_layers)]
    
    initializer = [trial.suggest_categorical(
        f'initializer_{i}', ['lecun_normal', 'he_normal', 'he_uniform', 'glorot_normal', 'glorot_uniform']) for i in range(num_layers)]

    kernel_sizes = [trial.suggest_int(
        f'kernel_size{i}', 2, 24) for i in range(num_layers)]
    
    beta = trial.suggest_float('beta', 0.1, 1.0)

    poolsize = [[16],
                [4, 4],
                [2, 2, 4],
                [2, 2, 2, 2]]
    
    n_clusters = trial.suggest_int('n_clusters', 5, 20)

    # Encoder
    encoder = Sequential(name='encoder')
    encoder.add(InputLayer(input_shape=input_shape))
    for i in range(num_layers):
        encoder.add(MaxPool2D(poolsize[num_layers - 1][i]))
        encoder.add(Conv2D(
            filters[i],
            kernel_sizes[i],
            strides=1,
            activation=activation[i],
            padding='same',
            kernel_initializer=initializer[i]
        ))
    encoder.add(MaxPool2D(5))

    # Decoder
    decoder = Sequential(name='decoder')
    decoder.add(InputLayer(input_shape=encoder.output_shape[1:]))
    for i in range(num_layers - 1, -1, -1):
        decoder.add(Conv2DTranspose(
            filters[i],
            kernel_sizes[i],
            strides=poolsize[num_layers - 1][i],
            activation=activation[i],
            padding='same',
            kernel_initializer=initializer[i]
        ))
    decoder.add(Conv2DTranspose(
        input_shape[-1],
        kernel_sizes[0],
        strides=5,
        activation=activation[0],
        padding='same',
        kernel_initializer=initializer[0]
    ))

    k_means = KMeans(n_clusters=n_clusters)
    
    # Compile the model
    auto_encoder = Sequential([encoder, decoder], name="autoencoder")

    return auto_encoder, k_means, beta

def custom_loss(y_true, y_pred):
    


def objective(trial):
    # Clear clutter from previous sessions
    tf.keras.backend.clear_session()

    # Create model
    model, k_means, beta = create_model(trial)

    def custom_loss(y_true, y_pred):
        # Reconstruction loss
        reconstruction_loss = tf.reduce_mean(tf.square(y_true - y_pred))

        feature_extractor = Sequential([model.get_layer("encoder"), Flatten()])

        feat_shape = model.get_layer("encoder").output_shape[-1]
        feature_arr = feature_extractor.predict(sim_arr, verbose=2)
        k_means.fit(feature_arr)
        labels = k_means.predict(feature_arr)

        # Total loss
        total_loss = reconstruction_loss + beta * silhouette_score(feature_arr, labels)

        return total_loss
    
    model.compile(optimizer='adam', loss=custom_loss)

    # Train the model
    history = model.fit(train_set, train_set, epochs=10, batch_size=16,
                        validation_data=(valid_set, valid_set), verbose=0)
    
    
    return history.history['val_loss'][-1]


# Perform hyperparameter optimization
study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name='autoencoder_DEC',
                            storage='sqlite:///test.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=100)


# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)
print("Best value:", study.best_value)

[I 2024-04-04 21:35:58,578] Using an existing study with name 'autoencoder_DEC' instead of creating a new one.
2024-04-04 21:35:59.154380: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 147916800 exceeds 10% of free system memory.
[W 2024-04-04 21:36:01,257] Trial 9 failed with parameters: {'num_layers': 3, 'filters_0': 8, 'filters_1': 11, 'activation_0': 'tanh', 'activation_1': 'selu', 'activation_2': 'selu', 'initializer_0': 'lecun_normal', 'initializer_1': 'he_uniform', 'initializer_2': 'glorot_normal', 'kernel_size0': 3, 'kernel_size1': 20, 'kernel_size2': 22, 'beta': 0.4127246977882997, 'n_clusters': 19} because of the following error: RuntimeError('Method requires being in cross-replica context, use get_replica_context().merge_call()').
Traceback (most recent call last):
  File "/home/tux/optim/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  Fil

RuntimeError: Method requires being in cross-replica context, use get_replica_context().merge_call()

In [ ]:
from keras.layers import Input, Dense, Layer
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K

# Define the autoencoder architecture
input_img = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(x_train, x_train, epochs=100, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Define the clustering layer
class ClusteringLayer(Layer):
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights

    def build(self, input_shape):
        input_dim = input_shape[1]
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

# Define the number of clusters
n_clusters = 10

# Add clustering layer on top of the encoder
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoded)

# Combine the autoencoder and clustering layer into a single model
model = Model(inputs=input_img, outputs=[decoded, clustering_layer])

# Define loss function
def custom_loss(y_true, y_pred):
    reconstruction_loss = K.mean(K.square(y_true - y_pred), axis=-1)
    clustering_loss = K.mean(K.square(y_pred - cluster_centers))
    return reconstruction_loss + clustering_loss

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss=[custom_loss, None])

# Train the model
model.fit(x_train, [x_train, y], epochs=100, batch_size=256, shuffle=True)
